<a href="https://www.kaggle.com/code/lxytypk/4fill-chinese-blanks?scriptVersionId=227171091" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### 1 定义数据集

In [1]:
import torch
from datasets import load_dataset

class DataSet(torch.utils.data.Dataset):
    def __init__(self,split):
        dataset=load_dataset(path='lansinuote/ChnSentiCorp',split=split)

        def f(data):
            return len(data['text'])>30

        self.dataset=dataset.filter(f)
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,i):
        text=self.dataset[i]['text']
        return text

dataset=DataSet('train')
len(dataset),dataset[0]

dataset_infos.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

(…)-00000-of-00001-02f200ca5f2a7868.parquet:   0%|          | 0.00/2.16M [00:00<?, ?B/s]

(…)-00000-of-00001-405befbaa3bcf1a2.parquet:   0%|          | 0.00/276k [00:00<?, ?B/s]

(…)-00000-of-00001-5372924f059fe767.parquet:   0%|          | 0.00/275k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9600 [00:00<?, ? examples/s]

(9192,
 '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般')

### 2 加载字典和分词工具

In [2]:
from transformers import BertTokenizer

token=BertTokenizer.from_pretrained('bert-base-chinese')
token

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

### 3 数据加载器

In [3]:
def collate_fn(data):
    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    #把第15个词固定替换为mask
    '''将每个输入序列的第15个词替换为掩码标记（mask token），并将原始的第15个词保存为标签'''
    labels=input_ids[:,15].reshape(-1).clone() #将其转换为一维张量,创建一个副本以避免对原始数据的修改
    input_ids[:,15]=token.get_vocab()[token.mask_token] #将每个序列的第15个词替换为掩码标记（mask token）

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
print(token.decode(input_ids[0]))
print(token.decode(labels[0]))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape

574
[CLS] 说 明 书 简 陋 随 盘 带 的 软 件 没 有 详 [MASK] 说 明 做 什 么 用 的 装 的 时 候 都 不 [SEP]
细


(torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16]))

### 4 模型试算

In [4]:
from transformers import BertModel

pretrained=BertModel.from_pretrained('bert-base-chinese')

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

out.last_hidden_state.shape

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

torch.Size([16, 30, 768])

### 5 定义下游任务模型

In [5]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        '''
        在掩码语言模型（Masked Language Model, MLM）任务中，模型需要预测被掩码的词。
        为了实现这一点，模型需要输出一个概率分布，表示每个词是被掩码词的可能性
        对这些分数进行 softmax 操作，可以得到每个词的概率分布
        '''
        self.decoder=torch.nn.Linear(768,token.vocab_size,bias=False) #输出维度为词汇表大小（token.vocab_size）
        self.bias=torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.decoder(out.last_hidden_state[:, 15])

        return out

model = Model()

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

torch.Size([16, 21128])

### 6 训练

In [6]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(5):
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader):
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)

        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 50 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            print(epoch, i, loss.item(), accuracy)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0 10.085941314697266 0.0
0 50 7.887359142303467 0.125
0 100 4.998171329498291 0.1875
0 150 5.772338390350342 0.0625
0 200 6.93461799621582 0.125
0 250 3.679710626602173 0.4375
0 300 4.257360935211182 0.5
0 350 2.7597968578338623 0.4375
0 400 3.6904923915863037 0.5625
0 450 2.877594470977783 0.5
0 500 3.2803633213043213 0.375
0 550 3.591104030609131 0.3125
1 0 2.5455546379089355 0.625
1 50 2.8495712280273438 0.5
1 100 2.6980535984039307 0.5625
1 150 2.3936049938201904 0.5625
1 200 2.37691593170166 0.625
1 250 1.721272349357605 0.6875
1 300 1.9356067180633545 0.6875
1 350 2.4535727500915527 0.625
1 400 2.8069353103637695 0.4375
1 450 2.192626714706421 0.625
1 500 2.0690088272094727 0.5625
1 550 2.0579609870910645 0.75
2 0 0.9846453070640564 0.875
2 50 1.4769757986068726 0.6875
2 100 1.2828673124313354 0.75
2 150 0.8742562532424927 0.8125
2 200 0.5200867056846619 0.875
2 250 1.9720672369003296 0.625
2 300 1.170210838317871 0.6875
2 350 1.882889747619629 0.6875
2 400 1.3706234693527222 0

### 7 测试

In [7]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=DataSet('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 15:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

        print(token.decode(input_ids[0]))
        print(token.decode(labels[0]), token.decode(out[0]))

    print(correct / total)


test()

Filter:   0%|          | 0/1200 [00:00<?, ? examples/s]

0
[CLS] 装 xp 驱 动 比 较 麻 烦 ， 我 还 有 一 个 [MASK] 知 设 备 识 别 不 了 ， 不 知 道 怎 么 [SEP]
未 告
1
[CLS] 定 的 商 务 大 床 房 ， 房 间 偏 小 了 ， [MASK] 过 经 济 性 酒 店 也 就 这 样 ； 环 境 [SEP]
不 不
2
[CLS] 不 知 道 是 不 是 夏 天 的 原 因 ， 有 点 [MASK] 。 网 卡 是 分 intel marvell realtek 的 [SEP]
热 烦
3
[CLS] 我 在 当 当 买 了 三 本 医 学 书 ， 这 本 [MASK] 最 差 的 ， 感 觉 作 者 是 个 江 湖 骗 [SEP]
是 是
4
[CLS] 优 点 ： （ 1 ） 房 间 还 可 以 ， 比 较 [MASK] 敞 。 （ 2 ） 洗 浴 不 错 ， 水 比 较 [SEP]
宽 宽
5
[CLS] 刚 去 的 时 候 有 点 难 找, 不 过 服 务 [MASK] 是 很 不 错 哦 ~ ~ 而 且 金 碧 辉 煌 [SEP]
真 还
6
[CLS] 很 一 般 ， 感 觉 到 主 人 公 内 心 很 肮 [MASK] ， 之 前 读 过 余 华 的 《 兄 弟 》 ， [SEP]
脏 脏
7
[CLS] [UNK] 屏 ， 亮 ， 可 视 角 度 大 （ 下 方 可 [MASK] 效 果 不 好 ） 。 较 轻 。 新 款 适 配 [SEP]
视 能
8
[CLS] 地 点 不 错, 离 开 汽 车 站 不 太 远, [MASK] 路 十 五 分 钟, 打 的 两 分 钟 就 可 [SEP]
走 马
9
[CLS] 一 如 既 往 的 支 持 ， 不 错 的 酒 店 ， [MASK] 务 上 再 上 去 一 点 就 好 了 。 宾 馆 [SEP]
服 服
10
[CLS] 这 套 书 三 册 ， 非 常 精 美 ， 内 容 也 [MASK] 好 ， 我 七 岁 的 女 儿 拿 到 书 后 ， [SEP]
很 很
11
[CLS] 早 晨 8 点 多, 已 经 没 什 么 吃 的, [MASK] 且 十 元 限 额 的. 餐 厅 服 务 员 态 [SEP]
而 而
12
[CLS] 钱 汇 过 去 半 个 月 ， 至 今 不 见 该 书 [MASK] 